<a href="https://colab.research.google.com/github/Andy7204/ML-Model-Implementation/blob/main/EMG_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

instead of nullifying the missing values replace them with the median of the values upto a decided number of record, say first clear upto x number, take median, then fill the rest with the same.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [ ]:
from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
path= '/content/Test_data2.csv'
df = pd.read_csv(path)
df.columns = ['0','1','2','3','4','5']
df.head()

,0,1,2,3,4,5
0,142.85,107.59,230.65,-165.58,-117.60,35.60
1,-194.01,-135.52,234.98,230.05,102.06,3.00
2,190.90,-105.37,-239.86,260.11,-104.57,-12.45
3,166.44,146.23,-228.16,-209.04,-126.93,-18.56
4,145.09,-138.30,245.46,-206.03,126.53,-16.54


In [ ]:
df.describe()

,0,1,2,3,4,5
count,343.000000,259.000000,278.000000,203.000000,399.000000,370.000000
mean,3.883440,-0.441274,7.721439,11.471034,5.342632,0.101541
std,297.121434,175.259969,320.706450,236.019270,135.433608,12.117341
min,-912.370000,-427.010000,-653.130000,-614.380000,-298.350000,-33.190000
25%,-203.395000,-152.965000,-271.897500,-195.780000,-120.640000,-8.395000
50%,122.730000,101.060000,203.175000,152.620000,101.160000,0.390000
75%,193.150000,146.190000,271.790000,200.360000,124.745000,7.602500
max,915.920000,376.870000,850.280000,533.070000,251.840000,35.600000


In [ ]:
max_length = max(df.apply(lambda x: x.count()))
df_filled = df.apply(lambda x: x.dropna().reindex(range(max_length)).fillna(pd.NA))
df_filled

,0,1,2,3,4,5
0,142.85,107.59,230.65,-165.58,-117.60,35.60
1,-194.01,-135.52,234.98,230.05,102.06,3.00
2,190.90,-105.37,-239.86,260.11,-104.57,-12.45
3,166.44,146.23,-228.16,-209.04,-126.93,-18.56
4,145.09,-138.30,245.46,-206.03,126.53,-16.54
...,...,...,...,...,...,...
394,NaN,NaN,NaN,NaN,-247.60,NaN
395,NaN,NaN,NaN,NaN,251.84,NaN
396,NaN,NaN,NaN,NaN,-158.39,NaN
397,NaN,NaN,NaN,NaN,107.40,NaN


In [ ]:
def remove_outliers_and_within_range(df, lower_percentile=25, upper_percentile=75):
    # Define a function to remove outliers using IQR
    def remove_outliers_iqr(series):
        Q1 = series.quantile(0.25)
        Q3 = series.quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 3 * IQR
        upper_bound = Q3 + 3 * IQR
        return series[((series >= lower_bound) & (series <= upper_bound))]

    # Define a function to remove data points within a certain range
    def remove_within_range(series):
        Q1 = series.quantile(lower_percentile / 100)
        Q3 = series.quantile(upper_percentile / 100)
        return series[(series < Q1) | (series > Q3)]

    # Remove outliers and data points within the specified range from each column
    for column in df.columns:
        df[column] = remove_outliers_iqr(remove_within_range(df[column]))

    return df

df_cleaned = remove_outliers_and_within_range(df_filled)
print("\nCleaned DataFrame:")
print(df_cleaned)


Cleaned DataFrame:
      0       1   2       3       4      5
0   NaN     NaN NaN     NaN     NaN  35.60
1   NaN     NaN NaN  230.05     NaN    NaN
2   NaN     NaN NaN  260.11     NaN -12.45
3   NaN  146.23 NaN -209.04 -126.93 -18.56
4   NaN     NaN NaN -206.03  126.53 -16.54
..   ..     ...  ..     ...     ...    ...
394 NaN     NaN NaN     NaN -247.60    NaN
395 NaN     NaN NaN     NaN  251.84    NaN
396 NaN     NaN NaN     NaN -158.39    NaN
397 NaN     NaN NaN     NaN     NaN    NaN
398 NaN     NaN NaN     NaN  128.05    NaN

[399 rows x 6 columns]


In [ ]:
df.describe()

,0,1,2,3,4,5
count,343.000000,259.000000,278.000000,203.000000,399.000000,370.000000
mean,3.883440,-0.441274,7.721439,11.471034,5.342632,0.101541
std,297.121434,175.259969,320.706450,236.019270,135.433608,12.117341
min,-912.370000,-427.010000,-653.130000,-614.380000,-298.350000,-33.190000
25%,-203.395000,-152.965000,-271.897500,-195.780000,-120.640000,-8.395000
50%,122.730000,101.060000,203.175000,152.620000,101.160000,0.390000
75%,193.150000,146.190000,271.790000,200.360000,124.745000,7.602500
max,915.920000,376.870000,850.280000,533.070000,251.840000,35.600000


In [ ]:
#df_cleaned.drop(columns= '5', axis =1)

In [ ]:
'''
cols = ['0','1','2','3','4']
low_q = df['5'].quantile(0.25)
high_q = df['5'].quantile(0.75)
temp_df = df_cleaned.drop(columns="5", axis=1)
for i, row in temp_df.iterrows():
  for col in cols:
    if row[col]<= high_q and row[col] >= low_q:
      row[col] = pd.NA
temp_df.describe()
'''

'\ncols = [\'0\',\'1\',\'2\',\'3\',\'4\']\nlow_q = df[\'5\'].quantile(0.25)\nhigh_q = df[\'5\'].quantile(0.75)\ntemp_df = df_cleaned.drop(columns="5", axis=1)\nfor i, row in temp_df.iterrows():\n  for col in cols:\n    if row[col]<= high_q and row[col] >= low_q:\n      row[col] = pd.NA\ntemp_df.describe()\n'

In [ ]:
#temp_df['5'] = df_cleaned['5']

In [ ]:
melted_df = pd.melt(df_cleaned, var_name='label', value_name='data')
print(melted_df)

     label  data
0        0   NaN
1        0   NaN
2        0   NaN
3        0   NaN
4        0   NaN
...    ...   ...
2389     5   NaN
2390     5   NaN
2391     5   NaN
2392     5   NaN
2393     5   NaN

[2394 rows x 2 columns]


In [ ]:
df_cleaned = melted_df.dropna(subset=['data'])

In [ ]:
# Negate negative values in the 'data' column using .loc
df_cleaned.loc[df_cleaned['data'] < 0, 'data'] *= -1
df_cleaned

,label,data
5,0,274.55
6,0,390.24
8,0,509.24
9,0,412.09
10,0,541.11
...,...,...
2359,5,11.54
2361,5,22.96
2362,5,19.73
2363,5,23.19


In [ ]:
df_cleaned = df_cleaned.sample(frac=1)
df_cleaned

,label,data
2156,5,25.00
339,0,273.86
986,2,317.97
2277,5,8.49
854,2,498.75
...,...,...
1047,2,276.51
459,1,166.82
1383,3,323.54
473,1,243.59


In [ ]:
X = df_cleaned.drop(columns=['label'])  # Features
y = df_cleaned['label']  # Labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# Further split the training data into training and validation sets (75% training, 25% validation)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Now you have X_train, X_test, X_val, y_train, y_test, y_val
# X_train, y_train: Training data and labels
# X_test, y_test: Testing data and labels
# X_val, y_val: Validation data and labels

print("Training set size:", X_train.shape[0])
print("Validation set size:", y_train.shape[0])
print("Testing set size:", X_val.shape[0])

Training set size: 488
Validation set size: 488
Testing set size: 163


In [ ]:
y_val = pd.to_numeric(y_val)
y_val.info()
y_train = pd.to_numeric(y_train)
y_train.info()
y_test = pd.to_numeric(y_test)
y_test.info()

<class 'pandas.core.series.Series'>
Index: 163 entries, 633 to 466
Series name: label
Non-Null Count  Dtype
--------------  -----
163 non-null    int64
dtypes: int64(1)
memory usage: 2.5 KB
<class 'pandas.core.series.Series'>
Index: 488 entries, 1741 to 126
Series name: label
Non-Null Count  Dtype
--------------  -----
488 non-null    int64
dtypes: int64(1)
memory usage: 7.6 KB
<class 'pandas.core.series.Series'>
Index: 279 entries, 14 to 2216
Series name: label
Non-Null Count  Dtype
--------------  -----
279 non-null    int64
dtypes: int64(1)
memory usage: 4.4 KB


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

param_grid_rf = {
    'n_estimators': [45, 47],
    'max_depth': [10,11],
}

# Define a custom scorer for precision, recall, and F1 score
scoring = {'precision': 'precision_macro', 'recall': 'recall_macro', 'f1': 'f1_macro'}

rf_model = RandomForestClassifier()
grid_search_rf = GridSearchCV(rf_model, param_grid_rf, cv=5, scoring=scoring, refit='f1')
grid_search_rf.fit(X_train, y_train)



GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [10, 11], 'n_estimators': [45, 47]},
             refit='f1',
             scoring={'f1': 'f1_macro', 'precision': 'precision_macro',
                      'recall': 'recall_macro'})

In [ ]:
print("Best parameters for Random Forest:", grid_search_rf.best_params_)
print("Best scores for Random Forest (Precision, Recall, F1):", grid_search_rf.best_score_)

Best parameters for Random Forest: {'max_depth': 10, 'n_estimators': 47}
Best scores for Random Forest (Precision, Recall, F1): 0.5435990912594372


In [ ]:
rf_precision = grid_search_rf.cv_results_['mean_test_precision'][grid_search_rf.best_index_]
rf_recall = grid_search_rf.cv_results_['mean_test_recall'][grid_search_rf.best_index_]
rf_f1 = grid_search_rf.cv_results_['mean_test_f1'][grid_search_rf.best_index_]
print("Random Forest Scores (Precision, Recall, F1):", rf_precision, rf_recall, rf_f1)

Random Forest Scores (Precision, Recall, F1): 0.540197507658034 0.5518055555555555 0.5435990912594372


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score


In [ ]:

y_test_pred_rf = grid_search_rf.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_test_pred_rf)
precision_rf = precision_score(y_test, y_test_pred_rf, average='macro')
recall_rf = recall_score(y_test, y_test_pred_rf, average='macro')
f1_rf = f1_score(y_test, y_test_pred_rf, average='macro')
accuracy_rf = accuracy_score(y_test, y_test_pred_rf)

print("RF Metrics (Accuracy, Precision, Recall, F1):", accuracy_rf, precision_rf, recall_rf, f1_rf)


RF Metrics (Accuracy, Precision, Recall, F1): 0.5806451612903226 0.5299920710824645 0.5258634280373411 0.5225391629646948


In [ ]:
#test_path = "/content/Test_data2.csv"
test_df = X
test_df.columns = ['data']
test_df

SyntaxError: invalid syntax (<ipython-input-32-912edebbaf13>, line 2)

In [ ]:
test_df.loc[test_df['data'] < 0, 'data'] *= -1
test_df

,data
2156,25.00
339,273.86
986,317.97
2277,8.49
854,498.75
...,...
1047,276.51
459,166.82
1383,323.54
473,243.59


In [ ]:
y_test1 = []
for i in range(160):
  y_test1.append(0)


In [ ]:
y_test_pred_data = grid_search_rf.predict(test_df)
accuracy_rf = accuracy_score(y_test1, y_test_pred_data)
precision_rf = precision_score(y_test1, y_test_pred_data, average='macro')
#recall_rf = recall_score(y_test1, y_test_pred_data, average='macro')
f1_rf = f1_score(y_test1, y_test_pred_data, average='macro')
accuracy_rf = accuracy_score(y_test1, y_test_pred_data)

print("RF Metrics (Accuracy, Precision, F1):", accuracy_rf, precision_rf, f1_rf)


RF Metrics (Accuracy, Precision, F1): 0.3875 0.3333333333333333 0.1861861861861862


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import classification_report

# Define the model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))  # Dropout layer to prevent overfitting
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))  # Dropout layer to prevent overfitting
model.add(Dense(6, activation='softmax'))  # Output layer with 6 units for 6 classes and softmax activation for multi-class classification

# Compile the model
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()
epochs = 20
batch_size = 64
# Train the model
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val), verbose=1)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

# Predict probabilities for each class
y_pred_prob = model.predict(X_test)

# Convert probabilities to class labels
y_pred = np.argmax(y_pred_prob, axis=1)

# Generate a classification report with numeric labels
print("Classification Report with Numeric Labels:")
print(classification_report(y_test, y_pred))


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                128       
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_5 (Dense)             (None, 6)                 198       
                                                                 
Total params: 2406 (9.40 KB)
Trainable params: 2406 (9.40 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/20
4/4 [======

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0])

In [ ]:
path = "/content/drive/MyDrive/Test.csv"
test_data = pd.read_csv(path)
test_data

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Test.csv'

In [ ]:
test_data.dropna(inplace=True)
test_data.info()

In [ ]:
test_data.describe()

In [ ]:
def remove_outliers_and_within_range(df, lower_percentile=25, upper_percentile=75):
    # Define a function to remove outliers using IQR
    def remove_outliers_iqr(series):
        Q1 = series.quantile(0.25)
        Q3 = series.quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 3 * IQR
        upper_bound = Q3 + 3 * IQR
        return series[((series >= lower_bound) & (series <= upper_bound))]

    # Define a function to remove data points within a certain range
    def remove_within_range(series):
        Q1 = series.quantile(lower_percentile / 100)
        Q3 = series.quantile(upper_percentile / 100)
        return series[(series < Q1) | (series > Q3)]

    # Remove outliers and data points within the specified range from each column
    for column in df.columns:
        df[column] = remove_outliers_iqr(remove_within_range(df[column]))

    return df


# Apply the function to the DataFrame
df_test_data = remove_outliers_and_within_range(test_data)

print("\nCleaned DataFrame:")
print(df_test_data)

In [ ]:
y_test_pred_rf = grid_search_rf.predict(X_test)
accuracy_rf = accuracy_score(Y_test, y_test_pred_rf)
precision_rf = precision_score(Y_test, y_test_pred_rf, average='macro')
recall_rf = recall_score(Y_test, y_test_pred_rf, average='macro')
f1_rf = f1_score(Y_test, y_test_pred_rf, average='macro')
accuracy_rf = accuracy_score(Y_test, y_test_pred_rf)

print("RF Metrics (Accuracy, Precision, Recall, F1):", accuracy_rf, precision_rf, recall_rf, f1_rf)


In [ ]:
print(Y_test)
print(y_test_pred_rf)